In [14]:
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from unidecode import unidecode

In [3]:
test_data = pd.read_csv("../../TestLLM/data/data.csv")

In [4]:
test_data

,Unnamed: 0,Question,Answers
0,58,What are the symptoms of Parkinson's Disease?,Treatment includes medications (levodopa) and ...
1,117,How is HIV/AIDS diagnosed?,Treatment includes antiretroviral therapy (ART...
2,166,How can I manage my Dengue Fever?,Diagnosis involves blood tests to detect the v...
3,278,Can Glaucoma be prevented?,Preventing Glaucoma involves regular eye exams...
4,261,Can Endometriosis be prevented?,Endometriosis is caused by the abnormal growth...
5,205,How can I manage my Multiple Sclerosis?,"Parkinson's Disease symptoms include tremors, ..."
6,135,How is Cancer diagnosed?,Symptoms vary depending on the type and stage ...
7,260,What treatment options are available for Endom...,Managing Endometriosis involves pain managemen...
8,5,How can I manage my Diabetes?,"Medication adherence, blood sugar monitoring."
9,162,How is Dengue Fever diagnosed?,"Lifestyle changes include avoiding alcohol, pr..."


In [15]:
load_dotenv()
URL_MODEL = os.getenv("URL_MODEL")

def getResponseFromMistralFinetuning(question):
    data = {"question": question}
    t1= time.time()
    response = requests.post(URL_MODEL, json=data, timeout=100.0)
    t2=time.time()
    if response.status_code == 200:
        return {"response":unidecode(response.text), "time":t2-t1}
    return response.text

In [16]:
def getResponse(question,reponse_attendue):
  response = getResponseFromMistralFinetuning(question)
  new_row={'Model':'Mistral-7B-Finetuning', 'Questions':question, 'Reponses':response['response'], 'Temps':response['time'], 'Reponses_Attendues':reponse_attendue}
  return new_row

In [17]:
list_data_mistral_finetuning=[]
for i in range(len(test_data)):
   list_data_mistral_finetuning.append(getResponse(test_data.iloc[i]['Question'],test_data.iloc[i]['Answers']))

In [18]:
df_mistral_finetuning = pd.DataFrame(data=list_data_mistral_finetuning)
df_mistral_finetuning.to_csv('../../TestLLM/data/mistral_finetuning_test.csv', index=False)

In [21]:
meditron_df = pd.read_csv('../../TestLLM/data/results_gpt35_meditron.csv')
meditron_df = meditron_df.loc[meditron_df['Model']=="Meditron"]
meditron_df

,Model,Questions,Reponses,Temps,Reponses_Attendues
10,Meditron,What are the symptoms of Parkinson's Disease?,What are the symptoms of Parkinson's Disease?\...,22.334797,Treatment includes medications (levodopa) and ...
11,Meditron,How is HIV/AIDS diagnosed?,How is HIV/AIDS diagnosed?\nHIV infection can ...,23.392662,Treatment includes antiretroviral therapy (ART...
12,Meditron,How can I manage my Dengue Fever?,How can I manage my Dengue Fever?\nHow can I m...,22.964650,Diagnosis involves blood tests to detect the v...
13,Meditron,Can Glaucoma be prevented?,Can Glaucoma be prevented? A qualitative study...,22.754100,Preventing Glaucoma involves regular eye exams...
14,Meditron,Can Endometriosis be prevented?,Can Endometriosis be prevented?\nThe most impo...,22.679845,Endometriosis is caused by the abnormal growth...
15,Meditron,How can I manage my Multiple Sclerosis?,"How can I manage my Multiple Sclerosis?"".\n\n#...",22.650548,"Parkinson's Disease symptoms include tremors, ..."
16,Meditron,How is Cancer diagnosed?,How is Cancer diagnosed?\nDuring a physical ex...,22.775442,Symptoms vary depending on the type and stage ...
17,Meditron,What treatment options are available for Endom...,What treatment options are available for Endom...,23.628584,Managing Endometriosis involves pain managemen...
18,Meditron,How can I manage my Diabetes?,How can I manage my Diabetes?\nWe’re here to h...,34.046091,"Medication adherence, blood sugar monitoring."
19,Meditron,How is Dengue Fever diagnosed?,How is Dengue Fever diagnosed?\nHow long does ...,24.812676,"Lifestyle changes include avoiding alcohol, pr..."


In [23]:
print(df_mistral_finetuning.iloc[0])

Model                                             Mistral-7B-Finetuning
Questions                 What are the symptoms of Parkinson's Disease?
Reponses              " The four main symptoms of Parkinsons disease...
Temps                                                         19.126295
Reponses_Attendues    Treatment includes medications (levodopa) and ...
Name: 0, dtype: object


In [25]:
print(df_mistral_finetuning['Temps'].mean())

3.9413936853408815


In [26]:
print(meditron_df['Temps'].mean())

24.203939509391784
